In [23]:
import ipyparallel as ipp
c = ipp.Client()
lbview1 = c.load_balanced_view()
lbview2 = c.load_balanced_view()

In [24]:
def returnsleep(x):
    import time
    time.sleep(x)
    return "that felt good"

In [28]:
import time
import itertools

job1 = {}
for i in range(4):
    job1[i] = lbview1.apply(time.sleep, 2)

job2 = {}
for i in range(4):
    with lbview1.temp_flags(after=job1[i]):
        job2[i] = lbview1.apply(time.sleep, 2)

job3 = {}
for i in range(4):
    with lbview1.temp_flags(after=job2[i]):
        job3[i] = lbview1.apply(time.sleep, 2)

job4 = {}
for i in range(4):
    with lbview1.temp_flags(after=job3[i]):
        job4[i] = lbview1.apply(returnsleep, 2)

        
waitfor = list(itertools.chain(*[i.msg_ids for i in job4.values()]))
print waitfor

with lbview1.temp_flags(after=waitfor):
    res = lbview1.apply(time.sleep, 2)

['a237079b-0737-477d-9a82-c84a66e827b6', '3a2663a0-defd-4dbc-a0ae-8fcfddc6b04e', '098eeee7-e64c-4efe-bcc2-beffefde3a3e', '9e3cad74-4756-4bc5-8da6-2b13598f64f9']


In [29]:
while 1:
    if not res.ready():
        print sum([job2[i].ready() for i in job2])
        print sum([job4[i].ready() for i in job4])
        time.sleep(1)
    else:
        break

0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
4
0
4
0
4
0
4
4
4
4


### Running sequential jobs w/ dependencies

In [14]:
c.wait_interactive()
c.

  17/17 tasks finished after   10 s
done


### Retrieving returned results

In [28]:
res = job4[0]
print res.result()
print res.successful()

that felt good
True


### Retrieving returned results AND raise error if failed

In [33]:
while 1:
    try:
        if all([job4[i].successful() for i in job4]):
            break
    except AssertionError:
        pass
    print 'hi', '[#####]', "time"
    time.sleep(1)

hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time


### Running a non-blocking wait call

In [22]:
while not all([job4[i].ready() for i in job4]):
    print 'hi', '[#####]', "time"
    time.sleep(1)

hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time
hi [#####] time


In [36]:
res.elapsed

20.041482

In [34]:
import numpy as np
from collections import Counter

a = np.array([0, 1, 2, 5, 3, 3, 3, 4])
dict(Counter(a))

{0: 1, 1: 1, 2: 1, 3: 3, 4: 1, 5: 1}

In [38]:
a
b = [0, 1, 2]

[i for i in a if i in b]

[0, 1, 2]

In [ ]:
### profiling
python -m cProfile -s cumulative ../ipyrad/assemble/cluster_within.py
